# Build bento model maintenance

The best model obtained in the notebook, which was the XGBoost model, will be build into a bento model, thats the reason of this notebook.

## Load libraries

The following libraries are needed to create the bento model

In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

import xgboost as xgb

import bentoml 

## Load the data

First lets load the data into a pandas DataFrame

In [2]:
data = 'https://raw.githubusercontent.com/FranciscoOrtizTena/ML_Zoomcamp/main/8_week/predictive_maintenance.csv'
df = pd.read_csv(data)

## Data preparation

Let's prepare the data as we did in the notebook file

In [3]:
df.columns = df.columns.str.lower().str.replace(' ','_')
df = df.drop(['udi', 'product_id'], axis=1)
df.failure_type = df.failure_type.str.lower().str.replace(' ','_')

Now let's split the data into the train and test sets.

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=2)
df_full_train = df_full_train.reset_index(drop=True)

y_full_train = df_full_train.target.values

del df_full_train['target']
del df_full_train['failure_type']

Making the one-hot encode using DictVectorizer

In [5]:
dicts_full_train = df_full_train.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

## XGBoost

Now let's train the XGBoost model

In [6]:
dfulltrain = xgb.DMatrix(X_full_train, label=y_full_train)
dtest = xgb.DMatrix(X_test)

In [7]:
xgb_params = {
    'eta': 0.3,
    'max_depth': 3,
    'min_child_weight': 10,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dfulltrain, num_boost_round=175)

## BentoML

Finally let's save the model into the bentoml.yaml file

In [13]:
bentoml.xgboost.save_model(
    'maintenance_predict_model',
    model,
    custom_objects={
        'dictVectorizer': dv
    },
    signatures={
        'predict': {
            'batchable': True,
            'batch_dim': 0,
        }
    }
)

Model(tag="maintenance_predict_model:asw4gns4q2vxgjv5", path="/Users/Frank/bentoml/models/maintenance_predict_model/asw4gns4q2vxgjv5/")